# Jinja2 and Ansible

This notebook demonstrates how jinja2 templates use loops and flow control and is combined with variables to generate expanded text.  Our example jinja2 template is loaded from a configuration file, as are the variables, which are stored in YAML format.

We are going to be generating a very simple chunk of logical-systems configuration.

Once we have worked through jinja2 syntax, we will run the ansible playbook: ```Section-5_ansible/apply_j2_config.yml``` to show it in action.  Let's take a quick look at those text files:

In [1]:
# Variables
!cat ./Section-5_ansible/group_vars/VMX.yml

variables:
  systems:
    RTR1:
      asn: 65534
      ibgp:
      - {ip: 192.168.1.2, router: rtr2}
      - {ip: 192.168.1.3, router: rtr3}
      - {ip: 192.168.1.5, router: rtr6}
      - {ip: 192.168.1.6, router: rtr7}


As you can see we have a nested dictionary in YAML format, containing all the variables we will later combine with a Jinja2 template.  Now let's look at the template:

In [2]:
# Jinja2 Template
!cat ./Section-5_ansible/logical_systems.j2

logical-systems { {% for system in systems %}
    {{ system }} {
        routing-options {
            autonomous-system {{ systems[system].asn }};
        }
        protocols { {% if systems[system].ibgp %}
            bgp {
                group ibgp {
                    type internal; {% for neighbour in systems[system].ibgp %}
                    neighbor {{ neighbour.ip }} { 
                        description {{ neighbour.router|upper }}; 
                    } {% endfor %}
                }
            } {% endif %}
        }
    } {% endfor %}
}

Examining some of the aspects of Jinja2:<ul>
    <li>Flow control and loop structures are enclosed in ```{% %}```.</li>
    <li>Variables are substituted by enclosing the variable name in ```{{  }}```.</li>
    <li>When accessing elements in a nested data structure the syntax is similar to python i.e. you enclose the item you are querying in ```[ ]```</li>
    <li>Custom filters e.g. ```|upper``` in this example can be used to transform data.  When working with strings you have filters available that are similar to string methods in Python</li>
    <li>You close for loops with ```{% endfor %}```</li>
    <li>You close if statements with ```{% endif %}```</li></ul>
    
    


Now lets put this in to practice.  We are going to use functions from the jinja2 and yaml libraries to load our templates and variables into a script and render them together to output a finished document:

In the example below, we perform the following steps:
<ol>
    <li>Import the jinja2 and yaml libraries</li>
    <li>Load the jinja2 template from a text file</li>
    <li>Instantiate a template object from the template text using the Template() class from the Jinja2 library</li>
    <li>Load variables from a YAML text file</li>
    <li>Render the template with the variables we obtained from the YAML file</li>
</ol>

In [1]:
import jinja2
import yaml

with open('./Section-5_ansible/group_vars/VMX.yml') as fh:
    yaml_vars = yaml.load(fh.read())
    
with open('./Section-5_ansible/logical_systems.j2') as fh:
    jinja2_template = jinja2.Template(fh.read())


print(jinja2_template.render(yaml_vars['variables']))

TypeError: load() missing 1 required positional argument: 'Loader'

In the next section we will use Ansible to push configuration to a Juniper device using these variables and templates.